In [ ]:
!pip install -q langchain==0.1.4

!pip install langchainhub==0.1.14
#!pip install --upgrade -q tiktoken==0.5.2
!pip install -q datasets==2.16.1
!pip install -q chromadb==0.4.22

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%pip install --upgrade --quiet  langchain-google-genai

In [ ]:
import pandas

In [ ]:
from datasets import load_dataset

data = load_dataset("keivalya/MedQuad-MedicalQnADataset", split='train')

In [ ]:
data = data.to_pandas()
data.head(10)

,qtype,Question,Answer
0,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,symptoms,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,exams and tests,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,treatment,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."
5,prevention,How to prevent Lymphocytic Choriomeningitis (L...,LCMV infection can be prevented by avoiding co...
6,information,What is (are) Parasites - Cysticercosis ?,Cysticercosis is an infection caused by the la...
7,susceptibility,Who is at risk for Parasites - Cysticercosis? ?,Cysticercosis is an infection caused by the la...
8,exams and tests,How to diagnose Parasites - Cysticercosis ?,"If you think that you may have cysticercosis, ..."
9,treatment,What are the treatments for Parasites - Cystic...,Some people with cysticercosis do not need to ...


In [ ]:
data=data[:80]

In [ ]:
data.shape

(80, 3)

In [ ]:
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma

In [ ]:
df_loader = DataFrameLoader(data, page_content_column="Answer")

In [ ]:
df_loader

In [ ]:
df_document = df_loader.load()

In [ ]:
import os
import getpass

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")


Enter your Google API Key: ··········


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
import chromadb
from chromadb.config import Settings

In [ ]:
db2 = Chroma.from_documents(df_document, embeddings, persist_directory="./chroma_db_med")

In [ ]:
from langchain.chains import RetrievalQA
retriever = db2.as_retriever()

In [ ]:
from langchain_google_genai import GoogleGenerativeAI

In [ ]:
llm = GoogleGenerativeAI(model="gemini-pro")

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=4, #Number of messages stored in memory
    return_messages=True #Must return the messages in the response.
)

In [ ]:
document_qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever
)

In [ ]:
document_qa.run("How to prevent Parasites - Cysticercosis ?")

'- Wash your hands with soap and warm water after using the toilet, changing diapers, and before handling food\n- Teach children the importance of washing hands to prevent infection\n- Wash and peel all raw vegetables and fruits before eating\n- Use good food and water safety practices while traveling in developing countries such as: \n      \n       - Drink only bottled or boiled (1 minute) water or carbonated (bubbly) drinks in cans or bottles\n       - Filter unsafe water through an "absolute 1 micron or less" filter AND dissolve iodine tablets in the filtered water; "absolute 1 micron" filters can be found in camping and outdoor supply stores'

Agent

The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.
  The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

  This is the chain responsible for deciding what step to take next. This is usually powered by a language model, a prompt, and an output parser.

Different agents have different prompting styles for reasoning, different ways of encoding inputs, and different ways of parsing the output. For a full list of built-in agents see agent types. You can also easily build custom agents, should you need further control.

In [ ]:
from langchain.agents import Tool, AgentExecutor

#Defining the list of tool objects to be used by LangChain.
tools = [
    Tool(
        name='Medical KB',
        func=document_qa.run,
        description=(
            'use this tool when answering medical knowledge queries to get '
            'more information about the topic'
        )
    )
]

Creating agent

In [ ]:
from langchain.agents import create_react_agent
from langchain import hub

prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(
    tools=tools,
    llm=llm,
    prompt=prompt
)

In [ ]:

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True,
                               memory=conversational_memory,
                               max_iterations=30,
                               max_execution_time=600,
                               handle_parsing_errors=True
                               )

In [ ]:
agent_executor.invoke({"input": "what is Huggingface?"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: Hugging Face is a platform that provides open-source tools for natural language processing (NLP). It offers a hub of pre-trained models, datasets, and metrics that can be used to train and evaluate NLP models. Additionally, Hugging Face provides a community forum where users can share knowledge and collaborate on NLP projects.

> Finished chain.


{'input': 'what is Huggingface?',
 'chat_history': [HumanMessage(content='what is LangChain?'),
  AIMessage(content='LangChain is a new blockchain platform that is designed to support the development of decentralized applications. It is based on the Tendermint consensus protocol and uses a novel sharding mechanism to improve scalability. LangChain also has a number of features that are designed to make it easy for developers to build and deploy decentralized applications. These features include a modular architecture, a rich set of APIs, and a developer-friendly programming language.')],
 'output': 'Hugging Face is a platform that provides open-source tools for natural language processing (NLP). It offers a hub of pre-trained models, datasets, and metrics that can be used to train and evaluate NLP models. Additionally, Hugging Face provides a community forum where users can share knowledge and collaborate on NLP projects.'}

In [ ]:
agent_executor.invoke({"input": "Do you know who is Clark Kent?"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: Clark Kent is the secret identity of Superman, a DC Comics superhero. He is a mild-mannered reporter for the Daily Planet newspaper, but when danger strikes, he transforms into the Man of Steel. Clark Kent was created by Jerry Siegel and Joe Shuster, and first appeared in Action Comics #1 in 1938.

> Finished chain.


{'input': 'Do you know who is Clark Kent?',
 'chat_history': [HumanMessage(content='what is LangChain?'),
  AIMessage(content='LangChain is a new blockchain platform that is designed to support the development of decentralized applications. It is based on the Tendermint consensus protocol and uses a novel sharding mechanism to improve scalability. LangChain also has a number of features that are designed to make it easy for developers to build and deploy decentralized applications. These features include a modular architecture, a rich set of APIs, and a developer-friendly programming language.'),
  HumanMessage(content='what is Huggingface?'),
  AIMessage(content='Hugging Face is a platform that provides open-source tools for natural language processing (NLP). It offers a hub of pre-trained models, datasets, and metrics that can be used to train and evaluate NLP models. Additionally, Hugging Face provides a community forum where users can share knowledge and collaborate on NLP projects

In [ ]:
agent_executor.invoke({"input": "What is her age?"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: The provided context does not mention Alice's age, so I cannot answer this question from the provided context.

> Finished chain.


{'input': 'What is her age?',
 'chat_history': [HumanMessage(content='what is LangChain?'),
  AIMessage(content='LangChain is a new blockchain platform that is designed to support the development of decentralized applications. It is based on the Tendermint consensus protocol and uses a novel sharding mechanism to improve scalability. LangChain also has a number of features that are designed to make it easy for developers to build and deploy decentralized applications. These features include a modular architecture, a rich set of APIs, and a developer-friendly programming language.'),
  HumanMessage(content='what is Huggingface?'),
  AIMessage(content='Hugging Face is a platform that provides open-source tools for natural language processing (NLP). It offers a hub of pre-trained models, datasets, and metrics that can be used to train and evaluate NLP models. Additionally, Hugging Face provides a community forum where users can share knowledge and collaborate on NLP projects.'),
  HumanMe

In [ ]:
 agent_executor.memory.clear()

In [ ]:
agent_executor.invoke({"input": """I have a patient that can have Botulism,
how can I confirm the diagnose?"""})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: What are the ways to confirm a Botulism diagnose?I do not know the answer to this question.Do I need to use a tool? No
Final Answer: Sorry, I am not able to answer this question. I recommend consulting a medical professional for more information.

> Finished chain.


{'input': 'I have a patient that can have Botulism,\nhow can I confirm the diagnose?',
 'chat_history': [],
 'output': 'Sorry, I am not able to answer this question. I recommend consulting a medical professional for more information.'}

In [ ]:
agent_executor.invoke({"input": "Is this an important illness?"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: Botulism is caused by a toxin produced by the bacterium Clostridium botulinum. It is a severe and potentially fatal illness that can cause paralysis and respiratory failure. The illness is usually associated with consumption of contaminated food and can also be caused by infection of wounds or by accidental inhalation of the toxin. Although botulism is uncommon, it is important to be aware of its symptoms and seek medical attention immediately if you suspect that you or someone you know has the illness.

> Finished chain.


{'input': 'Is this an important illness?',
 'chat_history': [HumanMessage(content='I have a patient that can have Botulism,\nhow can I confirm the diagnose?'),
  AIMessage(content='Sorry, I am not able to answer this question. I recommend consulting a medical professional for more information.')],
 'output': 'Botulism is caused by a toxin produced by the bacterium Clostridium botulinum. It is a severe and potentially fatal illness that can cause paralysis and respiratory failure. The illness is usually associated with consumption of contaminated food and can also be caused by infection of wounds or by accidental inhalation of the toxin. Although botulism is uncommon, it is important to be aware of its symptoms and seek medical attention immediately if you suspect that you or someone you know has the illness.'}

In [ ]:
agent_executor.invoke({"input": "Is it really important to know about it?"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: Yes, it is important to know about botulism because it is a potentially fatal illness. Early diagnosis and treatment are essential for improving the chances of a good outcome. If you suspect that you or someone you know has botulism, seek medical attention immediately.

> Finished chain.


{'input': 'Is it really important to know about it?',
 'chat_history': [HumanMessage(content='I have a patient that can have Botulism,\nhow can I confirm the diagnose?'),
  AIMessage(content='Sorry, I am not able to answer this question. I recommend consulting a medical professional for more information.'),
  HumanMessage(content='Is this an important illness?'),
  AIMessage(content='Botulism is caused by a toxin produced by the bacterium Clostridium botulinum. It is a severe and potentially fatal illness that can cause paralysis and respiratory failure. The illness is usually associated with consumption of contaminated food and can also be caused by infection of wounds or by accidental inhalation of the toxin. Although botulism is uncommon, it is important to be aware of its symptoms and seek medical attention immediately if you suspect that you or someone you know has the illness.')],
 'output': 'Yes, it is important to know about botulism because it is a potentially fatal illness. E

As can be seen on output we are loading 4 prompt and output associated with it, It means it can remember provious prompt for better user experience. We see these on chat gpt.